In [19]:
import sys
import time
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.layers import Input
from keras import backend as K
from keras import regularizers
from keras.layers import GaussianNoise, Conv2D, Conv2DTranspose, Activation, UpSampling2D
from keras.layers import Input, Flatten, BatchNormalization, Dense, Reshape
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
from keras.datasets import cifar10
%matplotlib inline

In [0]:
!pip install pydrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) 

In [0]:
#https://drive.google.com/open?id=1onwJKf6gNbTaHDHeW3_n2GHSLOx_VL2q
download = drive.CreateFile({'id': '1onwJKf6gNbTaHDHeW3_n2GHSLOx_VL2q'})
download.GetContentFile('mnist.npz')

In [85]:
class Generator:
    def __init__(self):
        self.input = Input([400])
        layer = Reshape((2,2,100))(self.input)
        #layer = Conv2DTranspose(1024,kernel_size= 4,strides = 4, padding = 'same')(layer)
        #layer = BatchNormalization(momentum=0.8)(layer)
        #layer = LeakyReLU(alpha=0.2)(layer)
        #layer = Conv2DTranspose(512,kernel_size= 4,strides = 2, padding = 'same')(layer)
        #layer = BatchNormalization(momentum=0.8)(layer)
        #layer = LeakyReLU(alpha=0.2)(layer)
        layer = Conv2DTranspose(256,kernel_size= 4,strides = 2, padding = 'same' )(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = LeakyReLU(alpha=0.2)(layer)
        layer = Conv2DTranspose(128,kernel_size= 4,strides = 2, padding = 'same')(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = LeakyReLU(alpha=0.2)(layer)
        layer = Conv2DTranspose(64,kernel_size= 4,strides = 2, padding = 'same' )(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = Activation("tanh")(layer)
        layer = Conv2DTranspose(3,kernel_size= 4,strides = 2, padding = 'same' )(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = Activation("tanh")(layer)
        #layer = Dense()
        self.model = Model(inputs=self.input, outputs=layer)
        #summary = self.model.summary()

In [41]:
G = Generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 400)               0         
_________________________________________________________________
reshape_12 (Reshape)         (None, 2, 2, 100)         0         
_________________________________________________________________
conv2d_transpose_28 (Conv2DT (None, 4, 4, 256)         409856    
_________________________________________________________________
batch_normalization_27 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose_29 (Conv2DT (None, 8, 8, 128)         524416    
_________________________________________________________________
batch_normalization_28 (Batc (None, 8, 8, 128)         512       
__________

In [86]:
class Discriminator:
    def __init__(self):
        # discriminator 
        #self.input = Input(shape=(28,28,1))
        #layer = Dense(784)(layer)
        self.input = Input([32**2 * 3])        
        layer = Reshape((32,32,3))(self.input)
        layer = Conv2D(64, kernel_size=4, strides=2, padding="same")(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = LeakyReLU(alpha=0.2)(layer)
        layer = Conv2D(128, kernel_size=4, strides=2, padding="same")(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = LeakyReLU(alpha=0.2)(layer)
        #layer = Dropout(0.25)(layer)
        layer = Conv2D(256, kernel_size=4, strides=2, padding="same")(layer)
        layer = BatchNormalization(momentum=0.8)(layer)
        layer = LeakyReLU(alpha=0.2)(layer)
        #layer = Dropout(0.25)(layer)
        #layer = Conv2D(512, kernel_size=4, strides=2, padding="same")(layer)
        #layer = BatchNormalization(momentum=0.8)(layer)
        #layer = LeakyReLU(alpha=0.2)(layer)
        #layer = Dropout(0.25)(layer)
        #layer = Conv2D(1024, kernel_size=4, strides=2, padding="same", activation = 'sigmoid')(layer)
        #layer = BatchNormalization(momentum=0.8)(layer)
        #layer = LeakyReLU(alpha=0.2)(layer)
        layer = Flatten()(layer)
        layer = Dense(250)(layer)
        self.feature = Model(inputs=self.input, outputs=layer)
        self.output = Dense(10, activation='sigmoid')(layer)
        self.model = Model(inputs=self.input, outputs=self.output)
        #summary = self.model.summary()

In [55]:
D = Discriminator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        3136      
_________________________________________________________________
batch_normalization_35 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 128)         131200    
_________________________________________________________________
batch_normalization_36 (Batc (None, 8, 8, 128)         512       
__________

In [87]:
def noise_gen(batch_size, z_dim):
    noise = np.zeros((batch_size, z_dim), dtype=np.float32)
    for i in range(batch_size):
        noise[i, :] = np.random.uniform(0, 1, z_dim)
    return noise

In [88]:
def plot_images(Generator, save2file=False, samples=25, step=0):
    filename = "./images/mnist_%d.png" % step
    #noise = np.random.normal(0, 1, (samples, 100))
    noise = np.random.uniform(-1.0, 1.0, size=[samples, 400])
    images = Generator.predict(noise)

    plt.figure(figsize=(10, 10))

    for i in range(images.shape[0]):
        plt.subplot(5, 5, i+1)
        #image = images[i, :, :, :]
        image = images[i, :]
        image = np.reshape(image, [28, 28])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()

    if save2file:
        plt.savefig(filename)
        plt.close('all')
    else:
        plt.show()

In [89]:
seed = 123
seed_data = 3
unlabeled_weight = 1
batch_size = 100
count =10

In [60]:
# load MNIST data
data = np.load('mnist.npz')
trainx = np.concatenate([data['x_train'], data['x_valid']], axis=0)
trainx_unl = trainx.copy()
trainx_unl2 = trainx.copy()
trainy = np.concatenate([data['y_train'], data['y_valid']]).astype(np.int32)
#nr_batches_train = int(trainx.shape[0] / args.batch_size)
nr_batches_train = int(trainx.shape[0] / batch_size)
testx = data['x_test']
testy = data['y_test'].astype(np.int32)
testy = np.reshape(testy, [testy.shape[0], 1])
#nr_batches_test = int(testx.shape[0] / args.batch_size)
nr_batches_test = int(testx.shape[0] / batch_size)

In [90]:
#Load cifar 10
(trainx, trainy), (testx, testy) = cifar10.load_data()
trainx = trainx.reshape(50000,32*32*3)
trainy = np.ravel(trainy)
testx = trainx.reshape(50000,32*32*3)
testy = np.ravel(testy)
trainx_unl = trainx.copy()
trainx_unl2 = trainx.copy()
#nr_batches_train = int(trainx.shape[0] / args.batch_size)
nr_batches_train = int(trainx.shape[0] / batch_size)
testx = data['x_test']
testy = data['y_test'].astype(np.int32)
testy = np.reshape(testy, [testy.shape[0], 1])
#nr_batches_test = int(testx.shape[0] / args.batch_size)
nr_batches_test = int(testx.shape[0] / batch_size)
trainx.shape, trainy.shape, testx.shape, testy.shape, nr_batches_train, nr_batches_test

((50000, 3072), (50000,), (10000, 784), (10000, 1), 500, 100)

In [91]:
trainy[1:10]

array([9, 9, 4, 1, 1, 2, 7, 8, 3], dtype=uint8)

In [92]:
# select labeled data
rng = np.random.RandomState(seed)
data_rng = np.random.RandomState(seed_data)
inds = data_rng.permutation(trainx.shape[0])
trainx = trainx[inds]
trainy = trainy[inds]
txs = []
tys = []
for j in range(10):
    txs.append(trainx[trainy == j][:count])
    tys.append(trainy[trainy == j][:count])
txs = np.concatenate(txs, axis=0)
tys = np.concatenate(tys, axis=0)

In [93]:
# set up tensorflow and keras
sess = tf.Session()
K.set_session(sess)
K.set_learning_phase(1)

In [94]:
# network
discriminator_model = Discriminator()
discriminator = discriminator_model.model
discriminator_feature = discriminator_model.feature
generator_model = Generator()
generator = generator_model.model

In [96]:
#### loss function computation
x_label = Input([32**2*3])
x_unlabel = Input([32**2*3])
labels = Input([1], dtype=tf.int32)
noise = Input([400])
fake_image = generator(noise)
output_before_softmax_label = discriminator(x_label)
output_before_softmax_unlabel = discriminator(x_unlabel)
output_before_softmax_fake = discriminator(fake_image)

In [97]:
z_exp_label = tf.reduce_mean(tf.reduce_logsumexp(output_before_softmax_label))
z_exp_unlabel = tf.reduce_mean(tf.reduce_logsumexp(output_before_softmax_unlabel))
z_exp_fake = tf.reduce_mean(tf.reduce_logsumexp(output_before_softmax_fake))
index_flattened = tf.range(0, batch_size) * output_before_softmax_label.shape[1] + tf.reshape(labels, [batch_size])
l_label = tf.gather(tf.reshape(output_before_softmax_label, [-1]), index_flattened)
l_unlabel = tf.reduce_logsumexp(output_before_softmax_unlabel)
loss_label = -tf.reduce_mean(l_label) + tf.reduce_mean(z_exp_label)
loss_unlabel = -0.5 * tf.reduce_mean(l_unlabel) + 0.5 * tf.reduce_mean(tf.nn.softplus(l_unlabel)) + \
               0.5 * tf.reduce_mean(tf.nn.softplus(tf.reduce_logsumexp(output_before_softmax_fake)))
loss_discriminator = tf.add(loss_label, tf.multiply(loss_unlabel, unlabeled_weight))

In [98]:
feature_generated = tf.reduce_mean(discriminator_feature(fake_image), axis=0)
feature_real = tf.reduce_mean(discriminator_feature(x_unlabel), axis=0)
loss_generator = tf.reduce_mean(tf.square(feature_generated - feature_real))

In [99]:
train_err = tf.reduce_mean(
    tf.to_float(tf.not_equal(tf.argmax(output_before_softmax_label, axis=1), tf.cast(tf.reshape(labels, [batch_size]), tf.int64))))
test_error = tf.reduce_mean(
    tf.to_float(tf.not_equal(tf.argmax(output_before_softmax_label, axis=1), tf.cast(tf.reshape(labels, [batch_size]), tf.int64))))

In [100]:
# train settings
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
discriminator_gradients = discriminator_optimizer.compute_gradients(loss_discriminator, discriminator.trainable_weights)
# discriminator_grads_and_vars = zip(discriminator_gradients, discriminator.trainable_weights)
discriminator_train = discriminator_optimizer.apply_gradients(discriminator_gradients)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [101]:
generator_optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
generator_gradients = generator_optimizer.compute_gradients(loss_generator, generator.trainable_weights)
# generator_grads_and_vars = zip(generator_gradients, generator.trainable_weights)
generator_train = generator_optimizer.apply_gradients(generator_gradients)

In [102]:
sess.run(tf.global_variables_initializer())

In [104]:
for epoch in range(2000):
    begin = time.time()

    # construct randomly permuted minibatches
    trainx = []
    trainy = []
    for t in range(int(trainx_unl.shape[0] / txs.shape[0])):
        inds = rng.permutation(txs.shape[0])
        trainx.append(txs[inds])
        trainy.append(tys[inds])
    trainx = np.concatenate(trainx, axis=0)
    trainy = np.concatenate(trainy, axis=0)
    trainy = np.reshape(trainy, [trainy.shape[0], 1])
    trainx_unl = trainx_unl[rng.permutation(trainx_unl.shape[0])]
    trainx_unl2 = trainx_unl2[rng.permutation(trainx_unl2.shape[0])]

    # train
    loss_label_record = 0.
    loss_unlabel_record = 0.
    train_err_record = 0.
    for t in range(nr_batches_train):
        noise_feed = noise_gen(batch_size, 400)
        _, loss_label_this, loss_unlabel_this, train_err_this = sess.run(
            [discriminator_train, loss_label, loss_unlabel, train_err], feed_dict={
                x_label: trainx[t * batch_size:(t + 1) * batch_size],
                x_unlabel: trainx_unl[t * batch_size:(t + 1) * batch_size],
                labels: trainy[t * batch_size:(t + 1) * batch_size],
                noise: noise_feed
            })

        loss_label_record += loss_label_this
        loss_unlabel_record += loss_unlabel_this
        train_err_record += train_err_this

        noise_feed = noise_gen(batch_size, 400)
        _, loss_generator_this = sess.run([generator_train, loss_generator], feed_dict={
            noise: noise_feed,
            x_unlabel: trainx_unl2[t * batch_size:(t + 1) * batch_size]
        })
    loss_label_record /= nr_batches_train
    loss_unlabel_record /= nr_batches_train
    train_err_record /= nr_batches_train

    # test
    test_err_record = 0.
    for t in range(nr_batches_test):
        test_err_this = sess.run(test_error, feed_dict={
            x_label: testx[t * batch_size:(t + 1) * batch_size],
            labels: testy[t * batch_size:(t + 1) * batch_size]
        })
        test_err_record += test_err_this
    test_err_record /= nr_batches_test

    # report
    if (epoch%5 == 0):
      print("Iteration %d, time = %ds, loss_lab = %.4f, loss_unl = %.4f, train err = %.4f, test err = %.4f" % (
          epoch, time.time() - begin, loss_label_record, loss_unlabel_record, train_err_record, test_err_record))
      plot_images(generator)
    sys.stdout.flush()

KeyboardInterrupt: 

In [0]:
plot_images(generator)